In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/huggingface/lerobot.git

In [ ]:
%cd lerobot

In [ ]:
!pip install -e .

In [ ]:
!pip install torchcodec

In [ ]:
# 2. Authenticate using your Kaggle Secret
from kaggle_secrets import UserSecretsClient
import os
user_secrets = UserSecretsClient()
os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")

from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])

In [ ]:
!pip install -U huggingface_hub transformers

In [ ]:
!python src/lerobot/scripts/lerobot_train.py \
    --dataset.repo_id=Sokrat2000/multiple_places \
    --policy.type=act \
    --policy.repo_id=Sokrat2000/multiple_places_network \
    --output_dir=outputs/train/multiple_places \
    --job_name=multiple_places \
    --policy.chunk_size=50 \
    --policy.n_action_steps=50 \
    --policy.device=cuda \
    --steps=100000 \
    --save_freq=20000 \
    --policy.push_to_hub=true \
    --wandb.enable=false

In [ ]:
import os
import shutil

# Path to where LeRobot saves checkpoints
ckpt_root = "/kaggle/working/lerobot/outputs/train/multiple_places/checkpoints"

# 1. Find all numerical folders (checkpoints)
folders = [f for f in os.listdir(ckpt_root) if f.isdigit()]

if folders:
    # 2. Sort them to find the highest number (the latest one)
    latest_step = sorted(folders, key=int)[-1]
    latest_folder_path = os.path.join(ckpt_root, latest_step)
    
    print(f"Found latest checkpoint at step: {latest_step}")
    
    # 3. Zip ONLY this folder
    output_zip = f"/kaggle/working/checkpoint_step_{latest_step}"
    shutil.make_archive(output_zip, 'zip', latest_folder_path)
    
    print(f"✅ Created: {output_zip}.zip")
    print("You can now download this single file from the Output tab.")
else:
    print("❌ No checkpoint folders found!")